In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import mlflow.sklearn

In [2]:
wine = pd.read_csv('winequality-red.csv')

X = wine.drop('quality', axis=1)
y = wine.quality
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [3]:
mlflow.set_experiment("experiments")
mlflow.sklearn.autolog()

In [4]:
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name = 'blend') as run:

    gbm = GradientBoostingRegressor(n_estimators=500, subsample=0.9, max_depth=12, learning_rate=0.1, random_state=42)
    svr = SVR(C=10.0, gamma=0.3, epsilon=0.0001)
    blender = VotingRegressor(estimators=[('gbm', gbm), ('svm', svr)],n_jobs=-2)
    blender.fit(X_train, y_train)

    preds = blender.predict(X_test)
    
    signature = infer_signature(X_train, blender.predict(X_train))
    mlflow.sklearn.log_model(blender, "model_blend", signature=signature)

    mae = mean_absolute_error(y_test, np.round(preds))

    mlflow.log_metric("mae", value = mean_absolute_error(y_test, np.round(preds)))

2022/11/27 22:10:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\ANL\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [ ]:
# code to load this model at any time to perform tests on it

import mlflow
logged_model = '/data/artifacts/1/2dc083e362bd419396ae1d05d92a06d1/artifacts/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))